
# 漢字・アルファベットからカタカナを推測するモデルを学習する
オープンソース辞書及び，Wikipediaから作成した漢字とカタカナのペアデータを用いて，Transformer+Seq2Seqのモデルで，スクラッチで学習する。

入力の文字（漢字・アルファベットなど）は一文字ずつに分割，また，出力の文字（カタカナ）も一文字ずつに分割して，学習する。



## データ準備
データは，[dict](../dict)で作成した姓名辞書データのoss.json及びseimei.jsonと，Wikipediaから抜き出した，漢字姓名とカタカナ姓名（スペース区切りのもの）を用いて，学習に使用した。単語単位である姓名辞書データと，Wikipediaのスペースで区切られた姓名を用いることで，姓名で漢字とカナが入力された場合の学習を可能とした。

また，ひらがなとカタカナ，


In [16]:
!date

2025年  6月  6日 金曜日 19:07:30 JST


In [17]:
# スクリプトで使う環境変数をセット
import os
os.environ['VER']='1.7o'
os.environ['DIR']='dataset/dataset.'+os.environ['VER']
os.environ['MODEL']='model/model.'+os.environ['VER']
os.environ["LOG"]='logs/logs.'+os.environ['VER']
os.environ['RATIO']="0.01"
os.environ['WIKI']="../dict/oss/wikipedia/wikiname.txt" # Wikipediaの概要欄から取得した姓名の漢字・アルファベットとカタカナのペアデータ。このファイルの姓名をスペースで区切って丹後他院位にしたものは，辞書ファイルに追加済み
os.environ['DICT']='../dict/data/data.'+os.environ['VER']
os.makedirs(os.environ['DIR'],exist_ok=True)
os.makedirs(os.environ['MODEL'],exist_ok=True)

## 漢字・アルファベットとカタカナのペアから学習する
学習は，訓練用データを用いて学習し，１エポックごとに開発用データ(valid.json)でLossを確認する。


### パラメタ
- emb_size 
入力，出力の文字のエンベッドのDimension

- nhead    
マルチヘッド数

- ffn_hid_dim    
FFNのDimension

- num_encoder_layer    
エンコードレイヤーの数

- num_decoder_layer    
デコードレイヤーの数

- lr     
学習率

- dropout    
ドロップアウトの割合, 0-1

- num_epochs    
何周学習データを用いて学習を行うか

- device    
mps,cpu,cudaから選択
  - mps        
   アップルシリコンを搭載したマシンで実行する際に選択
  - cuda         
   CUDAが利用できる環境で選択
  - cpu        
   上記以外は，CPUモードを選択

- earlystop_patient    
開発用データでlossが下がらなくなってearlystop_patient回計算が終了した場合に，num_epochs数以下でも，計算を終了させる

- output_dir    
モデルの出力ディレクトリ

- tensorboard_logdir    
tensorboard形式のログの出力ディレクトリ。学習状況を確認するためには`tensorboard --logdir logs`を実行後，ブラウザでhttp://localhost:6000/から確認
 



- prefix    
jsonl形式のデータのprefix

- source_lang    
jsonl形式のデータのsourceのキー

- target_lang    
jsonl形式のデータのtargetのキー

- train_file    
訓練用データのJSONLファイル

- valid_file    
開発用データのJSONLファイル

In [18]:
# 訓練実行

!python ./char_model.py \
  --emb_size 512 \
  --nhead 8 \
  --ffn_hid_dim 2048 \
  --batch_size 128 \
  --num_encoder_layers 8 \
  --num_decoder_layers 8 \
  --lr 0.00002 \
  --dropout 0.3 \
  --num_epochs 152 \
  --device cuda \
  --earlystop_patient 3 \
  --output_dir $MODEL \
  --tensorboard_logdir $LOG \
  --prefix translation \
  --source_lang kanji \
  --target_lang kana \
  --train_file $DIR/train.jsonl \
  --valid_file $DIR/valid.jsonl


num_epochs:152
Epoch: 1, Train loss: 3.283, Val loss: 2.572, Epoch time = 6332.798s
Epoch: 2, Train loss: 2.475, Val loss: 1.982, Epoch time = 6339.903s
Epoch: 3, Train loss: 2.094, Val loss: 1.663, Epoch time = 6328.976s
Epoch: 4, Train loss: 1.837, Val loss: 1.416, Epoch time = 6305.212s
Epoch: 5, Train loss: 1.631, Val loss: 1.236, Epoch time = 6295.878s
Epoch: 6, Train loss: 1.471, Val loss: 1.105, Epoch time = 6288.100s
Epoch: 7, Train loss: 1.347, Val loss: 1.001, Epoch time = 6284.781s
Epoch: 8, Train loss: 1.248, Val loss: 0.915, Epoch time = 6294.494s
Epoch: 9, Train loss: 1.167, Val loss: 0.851, Epoch time = 6317.882s
Epoch: 10, Train loss: 1.096, Val loss: 0.794, Epoch time = 6339.182s
Epoch: 11, Train loss: 1.035, Val loss: 0.739, Epoch time = 6339.323s
Epoch: 12, Train loss: 0.980, Val loss: 0.699, Epoch time = 6344.732s
Epoch: 13, Train loss: 0.932, Val loss: 0.665, Epoch time = 6305.883s
Epoch: 14, Train loss: 0.888, Val loss: 0.627, Epoch time = 6290.475s
Epoch: 15, Tra

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs.1.6.1o

## モデルを検証する
作成したモデルを，検証用データを用いて検証し，正解率を算出する


## パラメタ

- test_file
jsonl形式の検証用データファイル

- model_file
transformer_model.pyで作成したモデルファイル。

- outfile
検証結果を格納するファイル

- device
cpu限定

- nbest
検証用データの漢字姓名を入力データとした時に，モデルが出力するカタカナ姓名を確率の高い方からnbest個出力する。beam_width以下。searchパラメタがbothかbeamの時有効

- beam_width
ビームサーチのビーム幅。searchパラメタがbothかbeamの時有効

- max_len
出力するカタカナ姓名の最大長さ

- search
greedy,beam,bothから選択
-- greedy
貪欲法による探索
-- beam
ビームサーチによる探索
-- both
貪欲砲，ビームサーチ両方の探索



In [ ]:
# 検証用データを実行


!python generate_batch.py \
  --test_file $DIR/test.jsonl \
  --model_file $MODEL/checkpoint_best.pt \
  --outfile $DIR/generate.txt \
  --device cpu \
  --nbest 5 \
  --beam_width 5 \
  --max_len 100 \
  --search greedy





In [ ]:
# 検証結果を評価
import pandas as pd
import os
dr=os.environ["DIR"]
df = pd.read_csv(f'{dr}/generate.txt',encoding='utf-8',sep="\t")
tgt = df['tgt'].tolist()
pred = df['pred'].tolist()
ok=0
for t,p in zip(tgt,pred):
    if t == p:
        ok+=1

print(f'acc={ok/len(tgt)}')


# モデルをJavaのDJL用に変換する
Pytorchで学習したモデルをJavaで使えるようにモデルの変換を行う。



In [ ]:
# 漢字からカナ
!python convert_jitscript.py  \
    --model_file=$MODEL/checkpoint_best.pt \
    --model_script=$MODEL/script_$VER.pt \
    --encoder=$MODEL/encoder_$VER.pt \
    --decoder=$MODEL/decoder_$VER.pt \
    --positional_encoding=$MODEL/positional_encoding_$VER.pt \
    --generator=$MODEL/generator_$VER.pt \
    --src_tok_emb=$MODEL/src_tok_emb_$VER.pt \
    --tgt_tok_emb=$MODEL/tgt_tok_emb_$VER.pt \
    --vocab_src=$MODEL/vocab_src_$VER.txt \
    --vocab_tgt=$MODEL/vocab_tgt_$VER.txt \
    --params=$MODEL/params_$VER.json \
    --device=cpu



In [ ]:
!date


# カタカナから漢字・アルファベットを推測するモデルを学習する
オープンソース辞書及び，Wikipediaから作成した漢字とカタカナのペアデータを用いて，Transformerで，スクラッチで学習する。

入力の文字（カタカナ）は一文字ずつに分割，また，出力の文字（漢字・アルファベットなど）も一文字ずつに分割して，学習する。

データセットは，漢字からカタカナを推測するモデルのデータを，漢字とカナを逆にした者を作成して利用する。

## データ準備
データは，[dict](../dict)で作成した姓名辞書データのoss.json及びseimei.jsonと，Wikipediaから抜き出した，漢字姓名とカタカナ姓名（スペース区切りのもの）を用いて，学習に使用した。単語単位である姓名辞書データと，Wikipediaのスペースで区切られた姓名を用いることで，姓名で漢字とカナが入力された場合の学習を可能とした。

また，ひらがなとカタカナ，

In [ ]:
# スクリプトで使う環境変数をセット
import os

os.environ['DIR']='dataset/dataset.'+os.environ['VER']
os.environ['MODEL']='model/model_r.'+os.environ['VER']
os.environ["LOG"]='logs/logs_r.'+os.environ['VER']
os.environ['RATIO']="0.01"

os.makedirs(os.environ['DIR'],exist_ok=True)
os.makedirs(os.environ['MODEL'],exist_ok=True)


In [ ]:
# 訓練実行

!python ./char_model.py \
  --emb_size 512 \
  --nhead 8 \
  --ffn_hid_dim 2048 \
  --batch_size 32 \
  --num_encoder_layers 8 \
  --num_decoder_layers 8 \
  --lr 0.00002 \
  --dropout 0.3 \
  --num_epochs 145 \
  --device cuda  \
  --earlystop_patient 3 \
  --output_dir $MODEL \
  --tensorboard_logdir $LOG \
  --prefix translation \
  --source_lang kana \
  --target_lang kanji \
  --train_file $DIR/train.jsonl \
  --valid_file $DIR/valid.jsonl


In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

## モデルを検証する
作成したモデルを，検証用データを用いて検証し，正解率を算出する


## パラメタ

- test_file    
jsonl形式の検証用データファイル

- model_file    
transformer_model.pyで作成したモデルファイル。

- outfile    
検証結果を格納するファイル

- device    
cpu限定

- nbest    
検証用データの漢字姓名を入力データとした時に，モデルが出力するカタカナ姓名を確率の高い方からnbest個出力する。beam_width以下。searchパラメタがbothかbeamの時有効

- beam_width    
ビームサーチのビーム幅。searchパラメタがbothかbeamの時有効

- max_len    
出力するカタカナ姓名の最大長さ

- search    
greedy,beam,bothから選択    
  - greedy    
貪欲法による探索    
  - beam    
ビームサーチによる探索    
  - both    
貪欲砲，ビームサーチ両方の探索    

In [ ]:
# 検証用データを実行


!python generate_batch.py \
  --test_file $DIR/test.jsonl \
  --model_file $MODEL/checkpoint_best.pt \
  --outfile $DIR/generate_r.txt \
  --device cpu \
  --nbest 5 \
  --beam_width 5 \
  --max_len 100 \
  --search greedy


In [ ]:
# 検証結果を評価
import pandas as pd
import os
dr=os.environ["DIR"]
df = pd.read_csv(f'{dr}/generate.txt',encoding='utf-8',sep="\t")
tgt = df['tgt'].tolist()
pred = df['pred'].tolist()
ok=0
for t,p in zip(tgt,pred):
    if t == p:
        ok+=1

print(f'acc={ok/len(tgt)}')


# モデルをJavaのDJL用に変換する
Pytorchで学習したモデルをJavaで使えるようにモデルの変換を行う。


In [ ]:
# カナから漢字
!python convert_jitscript.py  \
    --model_file=$MODEL/checkpoint_best.pt \
    --model_script=$MODEL/script_$VER.pt \
    --encoder=$MODEL/encoder_$VER.pt \
    --decoder=$MODEL/decoder_$VER.pt \
    --positional_encoding=$MODEL/positional_encoding_$VER.pt \
    --generator=$MODEL/generator_$VER.pt \
    --src_tok_emb=$MODEL/src_tok_emb_$VER.pt \
    --tgt_tok_emb=$MODEL/tgt_tok_emb_$VER.pt \
    --vocab_src=$MODEL/vocab_src_$VER.txt \
    --vocab_tgt=$MODEL/vocab_tgt_$VER.txt \
    --params=$MODEL/params_$VER.json \
    --source_lang=kana \
    --target_lang=kanji \
    --device=cpu


In [ ]:
!date